In [ ]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf
import numpy as np
from sklearn import preprocessing
import folium as folium
from IPython.display import display
from folium import plugins

# Import the necessary modules
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [10, 6]
# Set up with a higher resolution screen (useful on Mac)
%config InlineBackend.figure_format = 'retina'

In [ ]:

data = pd.read_csv("../NYPD_CRIMES/NYPD_Arrests_Data__Historic_.csv", sep=",")
# data.dropna(inplace=True)
print(data.head(5))


In [ ]:
# data.isna().any() gives list of all columns containing NAN true/false
data.isna().sum()

In [ ]:
#data["OFNS_DESC"].value_counts()
print(data["OFNS_DESC"].value_counts().head(15))

def getLawCodesInOFNS_DESC(desc):
    lawCodes = data.loc[data["OFNS_DESC"] == desc]
    return lawCodes["LAW_CODE"]
    #TOTAL OF TOP 5 = 960111


print(getLawCodesInOFNS_DESC("DANGEROUS DRUGS").value_counts())



In [ ]:
# crimes = data.loc[data['AGE_GROUP'].isin(['18-24','45-64', '25-44', '<18', '65+'])]
crimes = data[["ARREST_DATE", "OFNS_DESC", "LAW_CODE", "AGE_GROUP", "Latitude", "Longitude"]]
crimes.dropna(inplace=True)


drugs = crimes.loc[crimes["OFNS_DESC"] == "DANGEROUS DRUGS"]
assults = crimes.loc[crimes["OFNS_DESC"] == "ASSAULT 3 & RELATED OFFENSES"]
robberies = crimes.loc[crimes["OFNS_DESC"] == "ROBBERY"]


mainCrimes = crimes.loc[crimes["OFNS_DESC"].isin(["DANGEROUS DRUGS", "ASSAULT 3 & RELATED OFFENSES", "ROBBERY"])]

print(drugs.head(1))
print("Amount of drug arrests",len(drugs))
print("Amount of assults arrests",len(assults))
print("Amount of robberies arrests",len(robberies))

main_datas = [drugs, assults, robberies]

In [ ]:
import datetime
import json

with open('Borough_Boundaries.geojson') as f:
    geojson_counties = json.load(f)

def printHeatmapTotal(dataset):    
    # Add years and months to dataset
    dataset['year']  = pd.DatetimeIndex(dataset['ARREST_DATE']).year
    dataset['month']  = pd.DatetimeIndex(dataset['ARREST_DATE']).month
    lista_tempo = []
    for x in dataset['month']:
        month_int = x
        lista_tempo.append(datetime.date(1900, month_int, 1).strftime('%B'))

    dataset['months_in_full'] = lista_tempo
    dataset['month_year'] = [d.split('/')[2] + " " + d.split('/')[0] for d in dataset.ARREST_DATE]
    6
    # Declare map
    INITIAL_COORDINATES = (40.696176, -73.948826)
    myMap2 = folium.Map(location=INITIAL_COORDINATES, zoom_start=10)
    
    weight_list = []
    dataset['counter'] = 1
    index_list = []
    
    # Add weighting
    for x in dataset['month_year'].sort_values().unique():
        index_list.append(x.replace(" ", "/"))
        weight_list.append(dataset.loc[dataset['month_year'] == x, ['Latitude', 'Longitude', 'counter']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

    
    # convert to (n, 2) nd-array format for heatmap
    stationArr = dataset[['Latitude', 'Longitude']]

    # plot heatmap
    plugins.HeatMap(stationArr, radius=15).add_to(myMap2)
        
    # choropleth
    folium.Choropleth(
        geo_data=geojson_counties,
        name='choropleth',
        fill_color='YlGn',
        fill_opacity=0,
        line_opacity=1,
        highlight=True
    ).add_to(myMap2)

    folium.LayerControl().add_to(myMap2)
    display(myMap2)

In [ ]:
for data in main_datas:
    printHeatmapTotal(data)

In [ ]:
import datetime

def heatmapTimelapseMonthsEachYear(dataset):    
    # Add years and months to dataset
    dataset['year']  = pd.DatetimeIndex(dataset['ARREST_DATE']).year
    dataset['month']  = pd.DatetimeIndex(dataset['ARREST_DATE']).month
    lista_tempo = []
    for x in dataset['month']:
        month_int = x
        lista_tempo.append(datetime.date(1900, month_int, 1).strftime('%B'))

    dataset['months_in_full'] = lista_tempo
    dataset['month_year'] = [d.split('/')[2] + " " + d.split('/')[0] for d in dataset.ARREST_DATE]
    
    # Declare map
    INITIAL_COORDINATES = (40.696176, -73.948826)
    myMap2 = folium.Map(location=INITIAL_COORDINATES, zoom_start=10)
    
    weight_list = []
    dataset['counter'] = 1
    index_list = []
    
    # Add weighting
    for x in dataset['month_year'].sort_values().unique():
        index_list.append(x.replace(" ", "/"))
        weight_list.append(dataset.loc[dataset['month_year'] == x, ['Latitude', 'Longitude', 'counter']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

    
    # Add to map
    hm = plugins.HeatMapWithTime(weight_list,auto_play=True, min_opacity=0.5,use_local_extrema=True, index=index_list, radius = 10
                                 ,gradient={0.1:"blue", 0.25: "green", 0.5:"yellow", 0.75: "orange", 1: "red"})
    hm.add_to(myMap2)
    display(myMap2)

In [ ]:
import datetime

def heatmapTimelapseMonthByYears(dataset, chosenMonth):    
    # Add years and months to dataset
    dataset['year']  = pd.DatetimeIndex(dataset['ARREST_DATE']).year
    dataset['month']  = pd.DatetimeIndex(dataset['ARREST_DATE']).month
    lista_tempo = []
    for x in dataset['month']:
        month_int = x
        lista_tempo.append(datetime.date(1900, month_int, 1).strftime('%B'))

    dataset['months_in_full'] = lista_tempo
    dataset['month_year'] = [d.split('/')[2] + " " + d.split('/')[0] for d in dataset.ARREST_DATE]
    
    # Declare map
    INITIAL_COORDINATES = (40.696176, -73.948826)
    myMap2 = folium.Map(location=INITIAL_COORDINATES, zoom_start=10)
    
    weight_list = []
    dataset['counter'] = 1
    index_list = []
    
    # Add weighting
    cont = 0
    for x in dataset['month_year'].sort_values().unique():
        if(x.split(" ")[1]) == chosenMonth: 
            index_list.append(x.replace(" ", "/"))
            weight_list.append(dataset.loc[dataset['month_year'] == x, ['Latitude', 'Longitude', 'counter']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())
    
    # Add to map
    hm = plugins.HeatMapWithTime(weight_list,auto_play=True, min_opacity=0.5,use_local_extrema=True, index=index_list, radius = 10
                                 ,gradient={0.1:"blue", 0.25: "green", 0.5:"yellow", 0.75: "orange", 1: "red"})
    hm.add_to(myMap2)
    display(myMap2)


In [ ]:
import datetime

def heatmapTimelapseByYear(dataset):    
    # Add years and months to dataset
    dataset['year']  = pd.DatetimeIndex(dataset['ARREST_DATE']).year
    dataset['month']  = pd.DatetimeIndex(dataset['ARREST_DATE']).month
    lista_tempo = []
    for x in dataset['month']:
        month_int = x
        lista_tempo.append(datetime.date(1900, month_int, 1).strftime('%B'))

    dataset['months_in_full'] = lista_tempo
    dataset['month_year'] = [d.split('/')[2] + " " + d.split('/')[0] for d in dataset.ARREST_DATE]
    
    # Declare map
    INITIAL_COORDINATES = (40.696176, -73.948826)
    myMap2 = folium.Map(location=INITIAL_COORDINATES, zoom_start=12)
    
    weight_list = []
    dataset['counter'] = 1
    index_list = []
    
    # Add weighting
    for x in dataset['year'].sort_values().unique():
        index_list.append(x)
        weight_list.append(dataset.loc[dataset['year'] == x, ['Latitude', 'Longitude', 'counter']].groupby(['Latitude', 'Longitude']).sum().reset_index().values.tolist())

    # Add to map
    hm = plugins.HeatMapWithTime(weight_list,auto_play=True, min_opacity=0.5,use_local_extrema=True, index=index_list, radius = 10
                                 ,gradient={0.1:"blue", 0.25: "green", 0.5:"yellow", 0.75: "orange", 1: "red"})
    hm.add_to(myMap2)
    display(myMap2)

In [ ]:
heatmapTimelapseMonthsEachYear(drugs)
heatmapTimelapseMonthByYears(drugs, "12")
heatmapTimelapseByYear(drugs)



In [ ]:
heatmapTimelapseMonthsEachYear(mainCrimes)